<a href="https://colab.research.google.com/github/imratankumar/compertive-study-of-nlp-model-LDA-BERT/blob/main/topic-modeling/BertTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/cleaned_df.csv')
df.drop(columns=['Unnamed: 0'])

,text,hashtags,mentions,topic
0,we get it youre not single during a pandemic ...,[],[],4
1,the best thing about the pandemic and quaranti...,[],[],4
2,taking it local for e of waco philadistilling...,"['waco', 'gin', 'quarantinelife']",['philadistilling'],2
3,can anyone please send me articles providing a...,['covid19'],[],5
4,which entity should the trudeau money tree pro...,"['covid19', 'coronavirus', 'canada', 'cdnpoli']",[],1
...,...,...,...,...
516665,nice day back east west hartfordct going to be...,[],[],2
516666,hi romi big fan back east how are things for ...,[],[],4
516667,isnt it funny that during this pandemic lefti...,[],['Sinatra_Says'],4
516668,no covid and no mask on anybody thx god,[],[],4


In [ ]:
df['text'] =df.text.astype(str).str.replace('\d+', '')
df['text'] = df['text'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
data = df.text.values.tolist()

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install transformers==4.16.2

In [ ]:
!pip install -U huggingface-hub

In [ ]:
!pip install sentence-transformers==1.2.0

In [ ]:
from sentence_transformers import SentenceTransformer


In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
embeddings = model.encode(data, show_progress_bar=True)

In [ ]:
with open('embeddings.npy', 'wb') as f:
    np.save(f, embeddings)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/embeddings.npy', 'rb') as f:
   embeddings = np.load(f)

In [ ]:
len(embeddings)

In [ ]:
!pip install umap-learn

In [ ]:
import umap

In [ ]:
umap_embeddings = umap.UMAP(n_neighbors=30, 
                            n_components=5, 
                            metric='cosine').fit_transform(embeddings)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/umap_embeddings.npy', 'wb') as f:
    np.save(f, umap_embeddings)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/umap_embeddings.npy', 'rb') as f:
   umap_embeddings = np.load(f)

In [ ]:
embeddings

In [ ]:
umap_embeddings

In [ ]:
!pip install -U numpy

In [ ]:
!pip install hdbscan==0.8.27

In [ ]:
import hdbscan
cluster = hdbscan.HDBSCAN(min_cluster_size=20,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)

In [ ]:
import matplotlib.pyplot as plt

# Prepare data
umap_data = umap.UMAP(n_neighbors=20, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()

In [ ]:
docs_df = pd.DataFrame(data, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(data))

In [ ]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .Doc
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "Doc": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Topic,Size
0,-1,415088
76,75,7983
521,520,6777
261,260,6465
61,60,5036
542,541,2804
476,475,2668
530,529,2164
511,510,2135
253,252,2113


In [ ]:
top_n_words[5][:20]

[('demographic', 0.6262695365650645),
 ('zip', 0.5799299425083212),
 ('philly', 0.5466791790847731),
 ('code', 0.5142956313903507),
 ('new', 0.49823373401211746),
 ('cases', 0.496546806483143),
 ('todays', 0.4940316124981013),
 ('coronavirusupdates', 0.46603256945792887),
 ('stayathome', 0.35934145493158853),
 ('deaths', 0.2869129193196037),
 ('covid', 0.06337426621322684),
 ('disagree', 0.051927890911983736),
 ('newmusicfriday', 0.03848573913128437),
 ('spotifyplaylist', 0.03736416500921366),
 ('amarillo', 0.03505367440734718),
 ('spotify', 0.02663457892631966),
 ('platforms', 0.02643137659368737),
 ('coronacrisis', 0.024156922378834936),
 ('jail', 0.022418133912600915),
 ('texas', 0.019424597044659268)]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

for i in range(20):
    # Calculate cosine similarity
    similarities = cosine_similarity(tf_idf.T)
    np.fill_diagonal(similarities, 0)

    # Extract label to merge into and from where
    topic_sizes = docs_df.groupby(['Topic']).count().sort_values("Doc", ascending=False).reset_index()
    topic_to_merge = topic_sizes.iloc[-1].Topic
    topic_to_merge_into = np.argmax(similarities[topic_to_merge + 1]) - 1

    # Adjust topics
    docs_df.loc[docs_df.Topic == topic_to_merge, "Topic"] = topic_to_merge_into
    old_topics = docs_df.sort_values("Topic").Topic.unique()
    map_topics = {old_topic: index - 1 for index, old_topic in enumerate(old_topics)}
    docs_df.Topic = docs_df.Topic.map(map_topics)
    docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

    # Calculate new topic words
    m = len(data)
    tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m)
    top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)

topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is d

,Topic,Size
0,-1,415128
74,73,7983
501,500,6798
253,252,6465
59,58,5036
522,521,2804
459,458,2668
510,509,2184
491,490,2155
245,244,2113
